<a href="https://colab.research.google.com/github/Diksha-Arsule/Wad/blob/main/prac5CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this first cell to ensure gensim is installed.
!pip install -q gensim

In [ ]:
!pip install -q numpy

In [ ]:
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim.models import KeyedVectors

In [ ]:
!pip install numpy==1.26.4 gensim==4.3.2 --force-reinstall --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 33.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.2

In [ ]:
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim.models import KeyedVectors

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

In [ ]:
paragraph = """
Natural language processing is a field of artificial intelligence that helps computers understand human language.
It involves techniques for processing and analyzing large amounts of natural language data.
Applications include machine translation, sentiment analysis, and question answering systems.
Deep learning has improved many NLP tasks by learning rich representations of words and sentences.
Training effective models often requires large datasets and careful preprocessing.
Working with small examples is useful for learning and debugging models.
"""

In [ ]:
def split_to_sentences(text):
    raw_sentences = []
    for chunk in text.strip().split("\n"):
        for sent in chunk.split('.'):
            s = sent.strip()
            if s:
                raw_sentences.append(s)
    return raw_sentences

sentences = split_to_sentences(paragraph)
tokenized_sentences = [simple_preprocess(s) for s in sentences]
print("✅ Sentences:", sentences)
print("✅ Tokenized:", tokenized_sentences)

✅ Sentences: ['Natural language processing is a field of artificial intelligence that helps computers understand human language', 'It involves techniques for processing and analyzing large amounts of natural language data', 'Applications include machine translation, sentiment analysis, and question answering systems', 'Deep learning has improved many NLP tasks by learning rich representations of words and sentences', 'Training effective models often requires large datasets and careful preprocessing', 'Working with small examples is useful for learning and debugging models']
✅ Tokenized: [['natural', 'language', 'processing', 'is', 'field', 'of', 'artificial', 'intelligence', 'that', 'helps', 'computers', 'understand', 'human', 'language'], ['it', 'involves', 'techniques', 'for', 'processing', 'and', 'analyzing', 'large', 'amounts', 'of', 'natural', 'language', 'data'], ['applications', 'include', 'machine', 'translation', 'sentiment', 'analysis', 'and', 'question', 'answering', 'system

In [ ]:
# Fit keras tokenizer
texts_for_tokenizer = [" ".join(tokens) for tokens in tokenized_sentences]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_for_tokenizer)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

print("✅ Vocabulary Size:", vocab_size)
print("✅ Sample Word Index:", dict(list(word_index.items())[:10]))

✅ Vocabulary Size: 58
✅ Sample Word Index: {'and': 1, 'language': 2, 'of': 3, 'learning': 4, 'natural': 5, 'processing': 6, 'is': 7, 'for': 8, 'large': 9, 'models': 10}


In [ ]:
# Convert integer into sequenses
sequences = tokenizer.texts_to_sequences(texts_for_tokenizer)

In [ ]:
# Generate CBOW context-target pairs
def cbow_pairs_from_sequences(sequences, window_size=2):
    contexts, targets = [], []
    for seq in sequences:
        seq_len = len(seq)
        for idx, word in enumerate(seq):
            begin = idx - window_size
            end = idx + window_size + 1
            context = [seq[i] for i in range(begin, end) if 0 <= i < seq_len and i != idx]
            contexts.append(context)
            targets.append(word)
    return contexts, targets

window_size = 2
raw_contexts, raw_targets = cbow_pairs_from_sequences(sequences, window_size)

print("✅ Number of training pairs:", len(raw_targets))
print("✅ Example pair (context → target):", list(zip(raw_contexts, raw_targets))[:3])


✅ Number of training pairs: 73
✅ Example pair (context → target): [([2, 6], 5), ([5, 6, 7], 2), ([5, 2, 7, 11], 6)]


In [ ]:
# Pad contexts to fixed length = window size*2
context_len = window_size * 2
X = sequence.pad_sequences(raw_contexts, maxlen=context_len, padding='pre')
y_indices = np.array(raw_targets)
y = to_categorical(y_indices, num_classes=vocab_size)

print("✅ Context shape:", X.shape)
print("✅ Target shape:", y.shape)

✅ Context shape: (73, 4)
✅ Target shape: (73, 58)


In [ ]:
# Build CBOW Model
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=context_len))
model.add(Lambda(lambda x: K.mean(x, axis=1)))  # average embeddings
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
epochs = 120
batch_size = 16
model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2)

Epoch 1/120
5/5 - 1s - 272ms/step - accuracy: 0.0137 - loss: 4.0616
Epoch 2/120
5/5 - 0s - 10ms/step - accuracy: 0.1507 - loss: 4.0454
Epoch 3/120
5/5 - 0s - 10ms/step - accuracy: 0.2192 - loss: 4.0324
Epoch 4/120
5/5 - 0s - 10ms/step - accuracy: 0.3836 - loss: 4.0194
Epoch 5/120
5/5 - 0s - 10ms/step - accuracy: 0.4521 - loss: 4.0064
Epoch 6/120
5/5 - 0s - 9ms/step - accuracy: 0.4795 - loss: 3.9933
Epoch 7/120
5/5 - 0s - 10ms/step - accuracy: 0.5479 - loss: 3.9795
Epoch 8/120
5/5 - 0s - 9ms/step - accuracy: 0.5753 - loss: 3.9656
Epoch 9/120
5/5 - 0s - 10ms/step - accuracy: 0.5890 - loss: 3.9510
Epoch 10/120
5/5 - 0s - 10ms/step - accuracy: 0.6164 - loss: 3.9358
Epoch 11/120
5/5 - 0s - 11ms/step - accuracy: 0.6301 - loss: 3.9194
Epoch 12/120
5/5 - 0s - 11ms/step - accuracy: 0.6849 - loss: 3.9029
Epoch 13/120
5/5 - 0s - 10ms/step - accuracy: 0.7123 - loss: 3.8853
Epoch 14/120
5/5 - 0s - 10ms/step - accuracy: 0.7260 - loss: 3.8668
Epoch 15/120
5/5 - 0s - 13ms/step - accuracy: 0.7123 - los

In [ ]:
# Save word vectors to text files
out_path = '/content/vectors.txt'

weights = model.get_weights()[0]
with open(out_path, 'w', encoding='utf-8') as f:
    f.write(f"{vocab_size-1} {embedding_dim}\n")
    for word, idx in tokenizer.word_index.items():
        vec = weights[idx]
        vec_str = " ".join(map(str, vec.tolist()))
        f.write(f"{word} {vec_str}\n")

print(f"✅ Saved trained word vectors to {out_path}")

✅ Saved trained word vectors to /content/vectors.txt


In [ ]:
# Load vectors with Gensim and text similarity
cbow_output = KeyedVectors.load_word2vec_format(out_path, binary=False)
print("✅ Words loaded:", len(cbow_output.key_to_index))

for w in list(word_index.keys())[:6]:
    if w in cbow_output:
        print(f"🔹 Most similar to '{w}':", cbow_output.most_similar(positive=[w], topn=5))
    else:
        print(f"⚠️ Word '{w}' not in vectors")

✅ Words loaded: 57
🔹 Most similar to 'and': [('requires', 0.3832623064517975), ('techniques', 0.35777586698532104), ('amounts', 0.2688322961330414), ('representations', 0.26700496673583984), ('useful', 0.2625177502632141)]
🔹 Most similar to 'language': [('field', 0.5148504972457886), ('amounts', 0.38503381609916687), ('data', 0.35203322768211365), ('processing', 0.29516923427581787), ('computers', 0.2699984610080719)]
🔹 Most similar to 'of': [('processing', 0.48266181349754333), ('sentences', 0.4026563763618469), ('analyzing', 0.3544209897518158), ('words', 0.2902710437774658), ('artificial', 0.2890635132789612)]
🔹 Most similar to 'learning': [('nlp', 0.48413875699043274), ('many', 0.4351005256175995), ('has', 0.3265098035335541), ('by', 0.3219414949417114), ('rich', 0.23188014328479767)]
🔹 Most similar to 'natural': [('data', 0.4718211591243744), ('is', 0.4268065094947815), ('analyzing', 0.3748922646045685), ('understand', 0.3048621118068695), ('human', 0.3012043535709381)]
🔹 Most sim

In [ ]:
# Summary Stats
print("✅ Total sentences:", len(tokenized_sentences))
print("✅ Total words:", sum(len(s) for s in tokenized_sentences))
print("✅ Unique vocab:", vocab_size - 1)

✅ Total sentences: 6
✅ Total words: 73
✅ Unique vocab: 57
